In [12]:
import numpy as np
import cv2
import random

In [13]:
def drawing(label):
    result = np.zeros(label.shape, dtype=np.uint8)
    H, W = label.shape
    
    mapping = {0:0, 1:128, 2:255}
    
    for h in range(H):
        for w in range(W):
            result[h][w] = mapping[label[h][w]]
    
    return result

In [14]:
def get_label0(mean_list, feature):
    num_label = len(mean_list)
    
    label = 0
    min_diff = np.sum((feature - mean_list[0])**2)
    
    for i in range(1, num_label):
        temp_diff = np.sum((feature - mean_list[i])**2)
        if temp_diff < min_diff:
            label = i
            min_diff = temp_diff
            
    return label

In [15]:
def get_label(mean_list, feature):
    num_label = len(mean_list)
    
    label = 0
    min_diff = np.sum(np.absolute(feature - mean_list[0]))
    
    for i in range(1, num_label):
        temp_diff = np.sum(np.absolute(feature - mean_list[i]))
        if temp_diff < min_diff:
            label = i
            min_diff = temp_diff
            
    return label

In [16]:
def K_means(stacked_energy_matrix, category):
    H, W, C = stacked_energy_matrix.shape
    
    # initialize all pixel to label 0
    labels = np.zeros((H, W), dtype=int)
    
    # initialize k-means center by random choose k point
    center_list = []
    
    # random initialize
    for i in range(category):
        random_h = random.randint(0,H-1) 
        random_w = random.randint(0,W-1)
        center_list.append(stacked_energy_matrix[random_h][random_w])
    '''
    # initilalize by three specific point
    center_list.append(stacked_energy_matrix[H-1][W-1])
    center_list.append(stacked_energy_matrix[H//2][W//2])
    center_list.append(stacked_energy_matrix[0][0])
    '''
    label_has_changed = True
    epoch = 0 
    while label_has_changed:
        #print(epoch)
        if epoch==10:
            break
        label_has_changed = False
        
        # initialize center accumulator
        center_accumulator_list = []
        for center in center_list:
            center_accumulator_list.append(np.zeros(center.shape))
        counter = [0] * category  
        
        # do one iteration
        for h in range(H):
            for w in range(W):
                temp_label = get_label(center_list, stacked_energy_matrix[h][w])
                if temp_label != labels[h][w]:
                    labels[h][w] = temp_label
                    label_has_changed = True
                counter[labels[h][w]] += 1
                center_accumulator_list[labels[h][w]] += stacked_energy_matrix[h][w]
                
        # update centers    
        for i in range(category):
            center_list[i] = center_accumulator_list[i] / counter[i]
            
        epoch+=1
                    
    return labels

In [17]:
def matrix_normalized(matrix):
    return (matrix - np.min(matrix))/(np.max(matrix)-np.min(matrix))

In [18]:
def Law_conv(image):
    L3 = (1/6) * np.array([1,2,1]).reshape(1,3)
    E3 = (1/2) * np.array([-1,0,1]).reshape(1,3)
    S3 = (1/2) * np.array([1,-2,1]).reshape(1,3)
    basis_mask = [L3, E3, S3]
    
    result_list = []
    for kernel_index in range(9):
        result = np.zeros(image.shape)
        padded_image = cv2.copyMakeBorder(image,1,1,1,1,cv2.BORDER_REFLECT)
        
        kernel = np.matmul(np.transpose(basis_mask[int(kernel_index/3)]),basis_mask[kernel_index%3])
        print("Processing kernel: ", kernel_index)
        for i in range(1,padded_image.shape[0]-1):
            for j in range(1,padded_image.shape[1]-1):
                result[i-1][j-1] = np.sum(padded_image[i-1:i+2,j-1:j+2] * kernel)
                
        result_list.append(result)
    
    return result_list
    

In [19]:
def energy_computation(microstructure_arrays, window_size):
    result_list = []
    p = (window_size-1) // 2
    
    for microstructure_array in microstructure_arrays:
        result = np.zeros(microstructure_array.shape)
        padded_image = cv2.copyMakeBorder(microstructure_array,p,p,p,p,cv2.BORDER_REFLECT)
        
        for i in range(p,padded_image.shape[0]-p):
            for j in range(p,padded_image.shape[1]-p):
                result[i-p][j-p] = np.sum(padded_image[i-p:i+p+1,j-p:j+p+1])**2
                
        result_list.append(result)
                
    return result_list

In [20]:
def Law_method(image):
    print("Evaluating microstructure arrays...")
    microstructure_arrays = Law_conv(image)
    print("Evaluating energy arrays...")
    energy_arrays = energy_computation(microstructure_arrays, window_size=15)
    print("Normalizing energy arrays...")
    normalize_energy_array = []
    for energy_array in energy_arrays:
        normalize_energy_array.append(matrix_normalized(energy_array))
    stacked_energy_array = np.stack(normalize_energy_array, axis = 2)
    print("Doing K means...")
    labels = K_means(stacked_energy_array, category=3)
    print("Drawing segmentation image...")
    result_image = drawing(labels)
    
    return result_image

In [21]:
img = cv2.imread('sample1.jpg', cv2.IMREAD_GRAYSCALE)
result_image = Law_method(img)

cv2.imshow('My Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Evaluating microstructure arrays...
Processing kernel:  0
Processing kernel:  1
Processing kernel:  2
Processing kernel:  3
Processing kernel:  4
Processing kernel:  5
Processing kernel:  6
Processing kernel:  7
Processing kernel:  8
Evaluating energy arrays...
Normalizing energy arrays...
Doing K means...
Drawing segmentation image...


In [22]:
cv2.imwrite("result_random2.jpg", result_image)

True